In [1]:
import pandas as pd
from sqlalchemy import create_engine
import yaml

In [81]:
# Read csv
trade_df = pd.read_csv("./data/trade.csv", encoding="utf-8", parse_dates=['ISSUEDT'],
                       dtype={'HSCD': str, "QTY": float, 'BYRADDR2': str, 'SPLYADDR2': str})
trade_df.head()

,HSCD,QTY,QTYUNIT,AMT,CUR,USDUNTPRICE,UNTPRICE,ITEM,BYRORGNM1,BYRPRTNUM,...,BYRADDR2,SPLYORGNM1,SPLYPRTNUM,SPLYADDR1,SPLYADDR2,ISSUEDT,TOTQTY,TOTQTYUNIT,TOTAMT,TOTAMTCUR
0,7210491090,84000.0,KG,45360.00,USD,540.00,540.00,GI COIL,주식회사 포스코대우,1318555320,...,주식회사 포스코대우,POSCO,5068100017,"1 GOEDONG-DONG, NAM-GU",NaN,2018-02-27,565475000.0,KG,2.577329e+08,USD
1,7210491090,1463000.0,KG,821355.00,USD,585.00,585.00,GI COIL,주식회사 포스코대우,1318555320,...,주식회사 포스코대우,POSCO,5068100017,"1 GOEDONG-DONG, NAM-GU",NaN,2018-02-27,565475000.0,KG,2.577329e+08,USD
2,7210491090,600000.0,KG,398580.00,USD,664.30,664.30,HG COIL - POSCO,주식회사 포스코대우,1318555320,...,주식회사 포스코대우,POSCO,5068100017,"1 GOEDONG-DONG, NAM-GU",NaN,2018-02-27,565475000.0,KG,2.577329e+08,USD
3,7225509000,2398000.0,KG,416301.78,USD,501.15,501.15,CR COIL (0.5MM<=T<1.0MM),주식회사 포스코대우,1318555320,...,주식회사 포스코대우,POSCO,5068100017,"1 GOEDONG-DONG, NAM-GU",NaN,2018-02-27,565475000.0,KG,2.577329e+08,USD
4,7209169000,23100000.0,KG,8134262.00,USD,490.00,490.00,CR COIL (3MM<=T),주식회사 포스코대우,1318555320,...,주식회사 포스코대우,POSCO,5068100017,"1 GOEDONG-DONG, NAM-GU",NaN,2018-02-27,565475000.0,KG,2.577329e+08,USD


In [30]:
csv_company_list = trade_df['BYRORGNM1'].append(trade_df['SPLYORGNM1'], ignore_index=True).drop_duplicates().reset_index(drop=True)
csv_company_list.head

<bound method NDFrame.head of 0                    주식회사 포스코대우
1                   도레이케미칼 주식회사
2               도레이첨단소재주식회사 4공장
3                   도레이첨단소재주식회사
4               도레이첨단소재주식회사 2공장
5              도레이첨단소재주식회사 군산공장
6                 티씨케이텍스타일 주식회사
7                   티씨케이텍스타일(주)
8                  티씨케이텍스타일주식회사
9                   주식회사 에스엘라이팅
10                      (주)대성엘텍
11       LG INTERNATIONAL CORP.
12                      대원화성(주)
13                      (주)기흥텍스
14                    주식회사 엘지화학
15                      이엘케이(주)
16                 주식회사 지노인터내셔널
17                      한화토탈(주)
18                    (주)티케이케미칼
19                     (주)GS글로벌
20                   (주) 지아이엔텍스
21                  (주)토비스안양사업장
22                      (주)은성무역
23                     코리아아웃도어스
24                  주식회사 에스티라이프
25                   엘지이노텍 주식회사
26                    삼양이노켐주식회사
27                     동아베스텍(주)
28                     태광산업주식회사
29                      (주)약진통상
          

In [4]:
trade_df.columns

Index(['HSCD', 'QTY', 'QTYUNIT', 'AMT', 'CUR', 'USDUNTPRICE', 'UNTPRICE',
       'ITEM', 'BYRORGNM1', 'BYRPRTNUM', 'BYRADDR1', 'BYRADDR2', 'SPLYORGNM1',
       'SPLYPRTNUM', 'SPLYADDR1', 'SPLYADDR2', 'ISSUEDT', 'TOTQTY',
       'TOTQTYUNIT', 'TOTAMT', 'TOTAMTCUR'],
      dtype='object')

In [9]:
len(trade_df)

2492786

In [3]:
trade_df = trade_df[['HSCD', 'AMT', 'ITEM', 'BYRORGNM1', 'SPLYORGNM1', 'ISSUEDT', 'TOTQTY']]
trade_df.head()

,HSCD,AMT,ITEM,BYRORGNM1,SPLYORGNM1,ISSUEDT,TOTQTY
0,7210491090,45360.00,GI COIL,주식회사 포스코대우,POSCO,20180227,565475000.0
1,7210491090,821355.00,GI COIL,주식회사 포스코대우,POSCO,20180227,565475000.0
2,7210491090,398580.00,HG COIL - POSCO,주식회사 포스코대우,POSCO,20180227,565475000.0
3,7225509000,416301.78,CR COIL (0.5MM<=T<1.0MM),주식회사 포스코대우,POSCO,20180227,565475000.0
4,7209169000,8134262.00,CR COIL (3MM<=T),주식회사 포스코대우,POSCO,20180227,565475000.0


In [5]:
len(trade_df['BYRORGNM1'].unique())

16618

In [6]:
len(trade_df['HSCD'].unique())

7252

In [28]:
hscode_trade_table = trade_df.groupby(['HSCD', 'BYRORGNM1', 'SPLYORGNM1'])['AMT'].sum().reset_index()
hscode_trade_table.head()

,HSCD,BYRORGNM1,SPLYORGNM1,AMT
0,000000000,현대위아(주),(주)위테크,5.700000e+08
1,0000000000,LG전자 (주),백림화학(주),6.753112e+07
2,0000000000,LG전자 (주),성철사(주),4.347540e+07
3,0000000000,LG전자 (주),주)동양이엔피,2.157122e+08
4,0000000000,LG전자 (주),주)엠씨엠,1.312642e+09


In [47]:
# group by year
hscode_trade_table = trade_df.groupby(['BYRORGNM1', 'HSCD', trade_df.ISSUEDT.dt.month])['AMT'].sum().reset_index()
hscode_trade_table.head()

,BYRORGNM1,HSCD,ISSUEDT,AMT
0,YP무역,8519812210,2,7450000.0
1,영원무역 주식회사,1902301010,1,72436864.0
2,영원무역 주식회사,1902301010,2,271167580.0
3,지니제이 인터내셔널,1704902090,1,27991.6
4,지니제이 인터내셔널,1905901040,2,108110.0


In [48]:
hscode_trade_table.to_csv("trade_month.csv", mode='w', index=False)

In [6]:
hscode_trade_table.loc[hscode_trade_table['HSCD'] == '400911']
# print(' ')

,HSCD,BYRORGNM1,SPLYORGNM1,AMT
28160,400911,현대위아(주),(주)화승R&A,438440.0
28161,400911,현대위아(주),평화산업(주),5049400.0


In [22]:
hscode_trade_table.loc[hscode_trade_table['HSCD'] == '400911']['AMT'] * 100 \
    / hscode_trade_table.loc[hscode_trade_table['HSCD'] == '400911']['AMT'].sum()

28160     7.9893
28161    92.0107
Name: AMT, dtype: float64

In [7]:
# BUYER SHARE
trade_df.groupby(['BYRORGNM1', 'HSCD'])['AMT'].sum().reset_index().head()

,BYRORGNM1,HSCD,AMT
0,YP무역,8519812210,7450000.0
1,영원무역 주식회사,1902301010,343604444.0
2,지니제이 인터내셔널,1704902090,27991.6
3,지니제이 인터내셔널,1905901040,108110.0
4,케이앤씨(KNC),6004100000,19437.0


In [8]:
# SUPPLYER SHARE
trade_df.groupby(['SPLYORGNM1', 'HSCD'])['AMT'].sum().reset_index().head()

,SPLYORGNM1,HSCD,AMT
0,(주)알포메,3921130000,168365.0
1,(주)일호엔지니어링,9031809099,4500.0
2,"sindoh co.,ltd",8443311010,35918131.0
3,삼양발브종합메이커,8481400000,2282566.0
4,에이치알메디텍,8543709090,41080000.0


In [17]:
# SUPPLYER & BUYER MATCH
trade_df.groupby(['SPLYORGNM1', 'BYRORGNM1'])['AMT'].sum().reset_index()

,SPLYORGNM1,BYRORGNM1,AMT
0,(주)알포메,(주)세림티티시,1.683650e+05
1,(주)일호엔지니어링,주식회사 하나텍,4.500000e+03
2,"sindoh co.,ltd",파워탑,3.591813e+07
3,삼양발브종합메이커,지지아이 엔터프라이즈,2.282566e+06
4,에이치알메디텍,(주)지톤이에스바이오,4.108000e+07
5,우진코스메틱,주식회사코루제약,5.600000e+05
6,주식회사 파인수처스,주식회사코루제약,6.000000e+05
7,태웅산업,(주)케이케이텍스,2.620048e+07
8,( 주 ) 서흥알이에프,(주)신티에스,2.618456e+07
9,( 주 ) 세왕섬유,(주)새롬텍스타일,4.724970e+03


In [33]:
hscode_trade_table.loc[hscode_trade_table['SPLYORGNM1'] == '(주)알포메']

,HSCD,BYRORGNM1,SPLYORGNM1,AMT
10621,2929101000,흥진수지,(주)알포메,6749356.0
23852,3921130000,(주)세림티티시,(주)알포메,8098465.0


In [25]:
config = yaml.safe_load(open('dart_config.yml'))
db_url = 'mysql://' + config['DB_USER'] + ':' + config['DB_USER_PASSWORD'] + '@' + config['DB_HOST'] \
                            + '/' + config['DB_NAME'] + '?charset=utf8'
engine = create_engine(db_url, encoding='utf-8')
conn = engine.connect()

In [27]:
company_df = pd.read_sql_table(config['DB_TABLE_NAME_1'], con=conn, index_col='id', 
                          columns=['id', 'name', '업종', '상장일', '지역'])
company_df.head()

,name,업종,상장일,지역
id,,,,
1,롯데케미칼,기초 화학물질 제조업,1991-05-30,서울특별시
2,대한유화,기초 화학물질 제조업,1999-08-11,서울특별시
3,이화산업,기타 전문 도매업,1994-04-29,서울특별시
4,디케이앤디,플라스틱제품 제조업,2018-11-20,경기도
5,송원산업,기타 화학제품 제조업,1977-06-25,울산광역시


In [34]:
company_df.loc[company_df['name'] == trade_df.loc[1, 'BYRORGNM1']]

,name,업종,상장일,지역
id,,,,


In [35]:
account_df = pd.read_sql_table(config['DB_TABLE_NAME_2'], con=conn, index_col='id',
                         columns=['id', 'company_id', 'year', 'quarter', 'data_period', 'start_date', 'end_date',
                                 'error', '매출액', '매출원가', '매출총이익', '판매비와관리비', '영업이익', '기타수익',
                                 '기타비용', '지분법이익', '금융수익', '금융비용', '법인세비용차감전순이익', '법인세비용',
                                 '분기순이익'])
account_df.head()

,company_id,year,quarter,data_period,start_date,end_date,error,매출액,매출원가,매출총이익,판매비와관리비,영업이익,기타수익,기타비용,지분법이익,금융수익,금융비용,법인세비용차감전순이익,법인세비용,분기순이익
id,,,,,,,,,,,,,,,,,,,,
1,1,2012,1,quarter,2012-01-01,2012-03-31,None,385.037,354.284,30.75290,10.4784,21.96120,7.92900,NaN,NaN,3.57648,3.55188,22.51700,4.337470,18.17950
2,1,2012,2,quarter,2012-01-01,2012-06-30,None,394.880,388.575,6.30579,11.0771,-2.83395,6.12777,NaN,NaN,1.83494,2.85516,-3.29184,-1.875760,-1.41608
3,1,2012,3,quarter,NaT,NaT,None,413.187,382.764,30.42240,11.0637,20.22220,4.32802,NaN,NaN,2.78657,3.82426,20.31320,-4.577300,15.73590
4,1,2012,4,quarter,2012-01-01,2012-12-31,None,350.719,362.628,-11.90660,8.9963,-23.49350,NaN,NaN,NaN,3.53929,2.76902,-22.25280,19.772700,-15.59320
5,1,2013,1,quarter,2013-01-01,2013-03-31,None,417.124,394.622,22.50180,10.7584,11.74340,NaN,NaN,NaN,4.37646,6.42669,11.73790,0.372595,11.36530


In [58]:
trade_df['CUR'].drop_duplicates().reset_index()

,index,CUR
0,0,USD
1,2020,EUR
2,2100,JPY
3,11416,CNY
4,29998,INR
5,33745,VND
6,35613,SEK
7,35631,GBP
8,35800,KRW
9,48885,IDR


In [56]:
trade_df.groupby(['CUR']).agg('size').reset_index(name="counts")

,CUR,counts
0,ALL,33
1,AUD,140
2,CAD,56
3,CHF,191
4,CNY,3047
5,EUR,9360
6,GBP,161
7,HKD,6
8,IDR,2
9,INR,48


In [82]:
trade_df = trade_df[['HSCD', 'AMT', 'CUR', 'BYRORGNM1']]
trade_df.head()

,HSCD,AMT,CUR,BYRORGNM1
0,7210491090,45360.00,USD,주식회사 포스코대우
1,7210491090,821355.00,USD,주식회사 포스코대우
2,7210491090,398580.00,USD,주식회사 포스코대우
3,7225509000,416301.78,USD,주식회사 포스코대우
4,7209169000,8134262.00,USD,주식회사 포스코대우


In [83]:
len(trade_df)

2492786

In [85]:
trade_df = trade_df.loc[(trade_df['CUR'] == 'USD') | (trade_df['CUR'] == 'KRW')].reset_index(drop=True)
trade_df.head()

,HSCD,AMT,CUR,BYRORGNM1
0,7210491090,45360.00,USD,주식회사 포스코대우
1,7210491090,821355.00,USD,주식회사 포스코대우
2,7210491090,398580.00,USD,주식회사 포스코대우
3,7225509000,416301.78,USD,주식회사 포스코대우
4,7209169000,8134262.00,USD,주식회사 포스코대우


In [86]:
len(trade_df)

2447526

In [87]:
# apply currency
trade_df.loc[trade_df['CUR'] == 'USD', 'AMT'] = trade_df['AMT'] * 1186.69
trade_df.head()

,HSCD,AMT,CUR,BYRORGNM1
0,7210491090,5.382826e+07,USD,주식회사 포스코대우
1,7210491090,9.746938e+08,USD,주식회사 포스코대우
2,7210491090,4.729909e+08,USD,주식회사 포스코대우
3,7225509000,4.940212e+08,USD,주식회사 포스코대우
4,7209169000,9.652847e+09,USD,주식회사 포스코대우


In [68]:
trade_df.loc[trade_df['CUR'] == 'KRW'].head()

,HSCD,AMT,CUR,BYRORGNM1
35800,7219341090,1.002408e+08,KRW,주식회사 포스코대우
35801,7219341090,9.336568e+09,KRW,주식회사 포스코대우
35802,7219341090,2.605554e+09,KRW,주식회사 포스코대우
40986,7210491090,1.920699e+08,KRW,주식회사 포스코대우
40987,7210491090,7.422800e+07,KRW,주식회사 포스코대우


In [88]:
hscode_trade_table = trade_df.groupby(['BYRORGNM1', 'HSCD'])['AMT'].sum().reset_index()
hscode_trade_table.head()

,BYRORGNM1,HSCD,AMT
0,YP무역,8519812210,7.450000e+06
1,영원무역 주식회사,1902301010,3.436044e+08
2,지니제이 인터내셔널,1704902090,3.321735e+07
3,지니제이 인터내셔널,1905901040,1.282931e+08
4,케이앤씨(KNC),6004100000,2.306569e+07


In [89]:
hscode_trade_table.to_csv("trade_data.csv", mode='w', index=False)